In [2]:
#Coonect to the patstat-spring 2024 database


import os, time
import pandas as pd
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv

# Load ENV (adjust path if .env lives elsewhere)
load_dotenv(".env", override=True)

def get_conn():
    return psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASS"),
        cursor_factory=DictCursor,
    )

conn = get_conn()
conn.cursor().execute("SET search_path = patstats, public;")   
print(" Connected")




 Connected


In [3]:
#Perform a Chunk query(Good for RAM) since direct query is slow at the moment

CHUNK_SIZE = 50_000

def chunk_query(conn, query, chunk_size=CHUNK_SIZE):
    base = query.strip().rstrip(";")
    offset = 0
    while True:
        page = pd.read_sql_query(f"{base} OFFSET {offset} LIMIT {chunk_size}", conn)
        if page.empty:
            break
        yield page
        offset += chunk_size


EXPLORE tls201_appln

In [4]:


#Check the number of rows in tls201_appln
pd.read_sql("""
    SELECT COUNT(*) AS total_rows
      FROM patstats.tls201_appln;
""", conn)


#Preview the first 10 rows of tls201_appln
preview = pd.read_sql("""
    SELECT *
      FROM patstats.tls201_appln
     LIMIT 10;
""", conn)

display(preview)          

#check the column names
pd.read_sql("""
    SELECT column_name, data_type, is_nullable
      FROM information_schema.columns
     WHERE table_schema = 'patstats'          -- change if your schema differs
       AND table_name   = 'tls201_appln'      -- change table if needed
     ORDER BY ordinal_position;
""", conn)






C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1906448027.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""
C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1906448027.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preview = pd.read_sql("""


,appln_id,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_epodoc,appln_nr_original,ipr_type,receiving_office,...,earliest_publn_date,earliest_publn_year,earliest_pat_publn_id,granted_patent,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors
0,960061578,US,200662630426,P,9999-12-31,9999,,,PI,,...,9999-12-31,9999,0,N,960061578,960061578,1,0,0,0
1,960061579,WO,2010514438,W,9999-12-31,9999,,,PI,JP,...,9999-12-31,9999,0,N,960061579,960061579,1,0,0,0
2,1,EP,103094,A,2000-02-15,2000,EP20000103094,00103094,PI,,...,2000-09-20,2000,293253293,Y,8554171,1,6,79,1,4
3,2,EP,107845,A,1992-12-02,1992,EP20000107845,00107845,PI,,...,2000-08-02,2000,301548848,Y,27517085,2,8,62,2,6
4,3,EP,202556,A,2000-07-17,2000,EP20000202556,00202556,PI,,...,2001-01-24,2001,291964096,N,7915918,3,4,23,2,3
5,4,EP,300208,A,2000-01-13,2000,EP20000300208,00300208,PI,,...,2000-07-26,2000,292901055,Y,22889365,4,6,28,1,2
6,5,EP,310305,A,2000-11-20,2000,EP20000310305,00310305,PI,,...,2001-05-30,2001,292462637,Y,23788431,5,4,44,1,4
7,6,EP,310786,A,2000-12-05,2000,EP20000310786,00310786,PI,,...,2001-06-13,2001,291480983,Y,23804898,6,6,156,1,3
8,7,EP,962768,A,2000-09-27,2000,EP20000962768,00962768,PI,,...,2002-08-07,2002,292266775,Y,10861834,7,7,13,1,3
9,8,EP,970100,A,2000-10-26,2000,EP20000970100,00970100,PI,,...,2003-09-03,2003,291966922,Y,26344962,8,8,8,1,3


C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1906448027.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,column_name,data_type,is_nullable
0,appln_id,integer,NO
1,appln_auth,character,YES
2,appln_nr,character varying,NO
3,appln_kind,character,YES
4,appln_filing_date,date,YES
5,appln_filing_year,smallint,YES
6,appln_nr_epodoc,character varying,YES
7,appln_nr_original,character varying,YES
8,ipr_type,character,YES
9,receiving_office,character,YES


In [5]:
#Pull all 2010-2013 applications with a family ID

cur = conn.cursor(name="appln_stream")   
cur.itersize = 100_000                   # change batch size

cur.execute("""
    SELECT appln_id, appln_filing_year, appln_auth, internat_appln_id
      FROM patstats.tls201_appln
     WHERE internat_appln_id IS NOT NULL
       AND appln_auth IN ('CA','US')
       AND appln_filing_year BETWEEN 2010 AND 2013
     ORDER BY internat_appln_id, appln_id
""")

chunks = []
for batch in iter(lambda: cur.fetchmany(cur.itersize), []):
    df = pd.DataFrame(batch, columns=[c.name for c in cur.description])
    # 🔹  process or write df here 
    chunks.append(df)      # or handle one chunk at a time
    print(f" {len(df):,} rows  (running {sum(map(len, chunks)):,})")

all_intl = pd.concat(chunks, ignore_index=True)





 100,000 rows  (running 100,000)
 100,000 rows  (running 200,000)
 100,000 rows  (running 300,000)
 100,000 rows  (running 400,000)
 100,000 rows  (running 500,000)
 100,000 rows  (running 600,000)
 100,000 rows  (running 700,000)
 100,000 rows  (running 800,000)
 100,000 rows  (running 900,000)
 100,000 rows  (running 1,000,000)
 100,000 rows  (running 1,100,000)
 100,000 rows  (running 1,200,000)
 100,000 rows  (running 1,300,000)
 100,000 rows  (running 1,400,000)
 100,000 rows  (running 1,500,000)
 100,000 rows  (running 1,600,000)
 100,000 rows  (running 1,700,000)
 100,000 rows  (running 1,800,000)
 100,000 rows  (running 1,900,000)
 100,000 rows  (running 2,000,000)
 100,000 rows  (running 2,100,000)
 17,462 rows  (running 2,117,462)


In [6]:
all_intl.head(-1)



,appln_id,appln_filing_year,appln_auth,internat_appln_id
0,275468780,2010,CA,0
1,275540546,2010,CA,0
2,277513403,2010,CA,0
3,298705113,2010,CA,0
4,298705114,2010,CA,0
...,...,...,...,...
2117456,339622529,2011,US,909172730
2117457,415911256,2013,US,909172745
2117458,422423106,2013,US,909172824
2117459,353621460,2010,US,909172827


In [7]:
all_intl['appln_auth'].value_counts()


appln_auth
US    1949412
CA     168050
Name: count, dtype: int64

In [8]:
#KEEP ONLY FAMILIES WITH BOTH CA AND US FILLINGS

filtered = (
    all_intl
    .groupby('internat_appln_id')
    .filter(lambda g: {'CA','US'} <= set(g['appln_auth']))
)
print(f"Remaining rows: {len(filtered):,} (families: {filtered['internat_appln_id'].nunique():,})")
filtered.head(-1)


Remaining rows: 1,748,445 (families: 61,496)


,appln_id,appln_filing_year,appln_auth,internat_appln_id
0,275468780,2010,CA,0
1,275540546,2010,CA,0
2,277513403,2010,CA,0
3,298705113,2010,CA,0
4,298705114,2010,CA,0
...,...,...,...,...
2116985,449826645,2013,US,442422828
2117107,339211898,2010,CA,902369219
2117108,352141815,2010,US,902369219
2117109,404963050,2012,US,902369219


EXPLORE tls207_pers_appln

In [9]:

#Check the number of rows in tls201_appln
pd.read_sql("""
    SELECT COUNT(*) AS total_rows
      FROM patstats.tls207_pers_appln;
""", conn)


#Preview the first 10 rows of tls201_appln
preview = pd.read_sql("""
    SELECT *
      FROM patstats.tls207_pers_appln
     LIMIT 10;
""", conn)

display(preview)          

#check the column names
pd.read_sql("""
    SELECT column_name, data_type, is_nullable
      FROM information_schema.columns
     WHERE table_schema = 'patstats'          -- change if your schema differs
       AND table_name   = 'tls207_pers_appln'      -- change table if needed
     ORDER BY ordinal_position;
""", conn)


C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1008157278.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""
C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1008157278.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preview = pd.read_sql("""


,person_id,appln_id,applt_seq_nr,invt_seq_nr
0,1,1,1,0
1,1,7,1,0
2,1,46,1,0
3,1,775,1,0
4,1,1192,1,0
5,1,1198,1,0
6,1,1203,1,0
7,1,1257,1,0
8,1,1263,1,0
9,1,1264,1,0


C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\1008157278.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,column_name,data_type,is_nullable
0,person_id,integer,NO
1,appln_id,integer,NO
2,applt_seq_nr,smallint,NO
3,invt_seq_nr,smallint,NO


In [ ]:
#Get lead applicants (person_id)
appln_ids = tuple(filtered['appln_id'].unique())
leads = pd.read_sql_query(f"""
    SELECT appln_id, person_id
      FROM tls207_pers_appln
     WHERE applt_seq_nr = 1
       AND appln_id IN {appln_ids}
""", conn)
print(f"Lead rows: {len(leads):,}")
leads.head()




C:\Users\shresthn\AppData\Local\Temp\1\ipykernel_24132\788802266.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  leads = pd.read_sql_query(f"""
